# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,krasnokuts'k,50.0660,35.1628,12.20,77,35,10.41,UA,1699774301
1,1,dabat,12.9842,37.7650,17.45,51,39,2.31,ET,1699774302
2,2,arraial do cabo,-22.9661,-42.0278,22.77,91,23,5.83,BR,1699774302
3,3,udachny,66.4167,112.4000,-15.95,95,100,3.89,RU,1699774302
4,4,parras de la fuente,25.4167,-102.1833,13.02,88,100,3.37,MX,1699774303


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,kidal,18.4411,1.4078,22.94,12,0,4.38,ML,1699768120
246,246,reggane,26.7158,0.1714,21.37,30,0,4.48,DZ,1699768076
266,266,vila velha,-20.3297,-40.2925,24.97,93,0,4.12,BR,1699767917
302,302,siwa oasis,29.2041,25.5195,21.22,47,0,1.68,EG,1699768173
499,499,santa cruz cabralia,-16.2781,-39.0247,23.13,94,0,2.06,BR,1699768221


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
77,kidal,ML,18.4411,1.4078,12,
246,reggane,DZ,26.7158,0.1714,30,
266,vila velha,BR,-20.3297,-40.2925,93,
302,siwa oasis,EG,29.2041,25.5195,47,
499,santa cruz cabralia,BR,-16.2781,-39.0247,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kidal - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
siwa oasis - nearest hotel: فندق الكيلانى
santa cruz cabralia - nearest hotel: Campo Bahia Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
77,kidal,ML,18.4411,1.4078,12,No hotel found
246,reggane,DZ,26.7158,0.1714,30,No hotel found
266,vila velha,BR,-20.3297,-40.2925,93,Hotel Prainha
302,siwa oasis,EG,29.2041,25.5195,47,فندق الكيلانى
499,santa cruz cabralia,BR,-16.2781,-39.0247,94,Campo Bahia Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)